 **Create and Run a local RAG pipeline from scratch**

## 1. Document/text preprocessing and embedding creation

In [ ]:
import os
import requests

# Get PDF document path
pdf_path = "human-nutrition-text.pdf"

# Download
if not os.path.exists(pdf_path):
    print(f"[INFO] File doesn't exist, downloading...")

    # Enter the URL of the PDF
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    # THe Local filename to save the downloaded file
    filename = pdf_path

    # Send a GET request to the URL
    responce = requests.get(url)

    # Check if the request was successfull
    if responce.status_code == 200:
        # Open the file and save it
        with open(filename , 'wb') as file:
            file.write(responce.content)
        print(f"[INFO] The file has been download and saved as {filename}")
    else:
        print(f"[INFO] Falied to download the file . Status code {responce.status_code}")

else:
    print(f"File {pdf_path} exists.")


In [ ]:
import fitz
from tqdm import tqdm


def text_formaatter(text: str) -> str:
    """ Performs minor formatting on text """
    cleaned_text = text.replace("\n" , " ").strip()

    # Pottentially more text formating functions can go here
    return cleaned_text

def open_and_read_pdf(pdf_path:str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_text = []
    for page_number , page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formaatter(text=text)
        pages_and_text.append({"page_number" : page_number - 41 , 
                               "page_char_count": len(text) , 
                               "page_word_count": len(text.split(" ")) , 
                               "page_sentence_count_raw": len(text.split(". ")) , 
                               "page_token_count": len(text) / 4  , # 1 token = 4 character.
                               "text": text})
    
    return pages_and_text

pages_and_text = open_and_read_pdf(pdf_path=pdf_path)
pages_and_text[:2]


In [ ]:
import random
random.sample(pages_and_text , k = 4)

In [ ]:
import pandas as pd 

df = pd.DataFrame(pages_and_text)
df.head()

In [ ]:
df.describe()

**Further text processing (splitting pages into sentences)**

In [ ]:
from spacy.lang.en import English

nlp = English()

# Add a sentencizer pipeline
nlp.add_pipe("sentencizer")

# Create a document instance as an example
doc = nlp("This is a sentence. This is another sentenece. I like elephants.")
assert len(list(doc.sents)) == 3

# Print out our sentence split
list(doc.sents)

In [ ]:
for item in tqdm(pages_and_text):
    item['sentences'] = list(nlp(item["text"]).sents)

    # Make sure all sentences are string (the default type is a spacyy datatype)

    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the senteneces
    item["page_sentence_count_spacy"] = len(item["sentences"])


In [ ]:
random.sample(pages_and_text , k = 1)

In [ ]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

### Chucking our sentence together

In [ ]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# Create a function to split a list of texts recursively into chunk size

def split_list(input_list: list[str] , slice_size : int = num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i : i + slice_size] for i in range(0 , len(input_list) , slice_size)]

test_list = list(range(25))
split_list(test_list)


In [ ]:
# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_text):
    item["sentence_chunks"] = split_list(input_list=item["sentences"] , slice_size=num_sentence_chunk_size)

    item["num_chunks"] = len(item["sentence_chunks"])

In [ ]:
random.sample(pages_and_text , k = 1)

In [ ]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

### Splitting each chunk into its own item

In [ ]:
import re

# Split each chunk into own item
pages_and_chunks = []
for item in tqdm(pages_and_text):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka join the list of sentences into one paragraph
        joined_sentence_chunk = "".join(sentence_chunk).replace("  " , " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])' , r'. \1' , joined_sentence_chunk) # ".A" => ". A" (will work for any capital letter)

        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get some stats on our chunks
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 chars

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

In [ ]:
random.sample(pages_and_chunks , k = 1)

In [ ]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

### Filter chunks of text for short chunks

In [18]:
min_token_length = 30

In [ ]:
# Filter our DataFrame for rows with under 30 tokens
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient = "records")
pages_and_chunks_over_min_token_len[:2]

In [ ]:
random.sample(pages_and_chunks_over_min_token_len , k = 1)

## Embedding our text chunks

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path = "all-mpnet-base-v2" , device = "mps")

# Create a list of sentences
sentences = ["The Sentence Transformer library provides an easy way to create embeddings.",
"Sentences can be embedded one by one or in a list.",
"I like horses!"]

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences , embeddings))

# See the embeddings
for sentence , embedding in embeddings_dict.items():
    print(f"Sentences: {sentences}")
    print(f"Embedding: {embedding}")
    print("")

In [ ]:
embeddings[0].shape

In [ ]:
%%time

embedding_model.to("mps")

# Embed each chunk one by one
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

In [ ]:
%%time

text_chunks = [item['sentence_chunk'] for item in pages_and_chunks_over_min_token_len]
text_chunks[523]

In [ ]:
len(text_chunks)

In [ ]:
%%time

# Embed all texts in batches
text_chunks_embeddings = embedding_model.encode(text_chunks , batch_size=32 ,convert_to_tensor=True)

text_chunks_embeddings


# Save embeddings to file


In [ ]:
pages_and_chunks_over_min_token_len[412]

In [52]:
# Save embeddings to file 
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path , index=False)

In [ ]:
# Import saved file and view
embeddings_df_save_path = "/Users/pandhari/Desktop/NLP/RAG/text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df_load.head()

# RAG - Search and Answer

### Similarity search

In [ ]:
import random
import torch
import numpy as np
import pandas as pd

device = "mps"

# Import texts and embedding df
text_chunks_and_embeddings_df = pd.read_csv("/Users/pandhari/Desktop/NLP/RAG/text_chunks_and_embeddings_df.csv")

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient= "records")

# Convert emedding column back to np.array (it got converted to string when it saved to CSV)
text_chunks_and_embeddings_df["embedding"] = text_chunks_and_embeddings_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]") , sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: Numpy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embeddings_df["embedding"].tolist()) , dtype=torch.float32).to(device=device)
embeddings.shape


In [ ]:
# Create model
from sentence_transformers import util , SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2" , device=device)

In [ ]:
# 1. Define the query
query = "good foods for protein"
print(f"Query: {query}")

# 2. Embed the query
# Note: it's important to embed you query with the same model you embedding your passages
query_embedding = embedding_model.encode(query , convert_to_tensor=True)
query_embedding = query_embedding.to(device=device)

# 3. Get Similaity socre with the dot product (use cosine similarity if outputs of model aren't normalize)
from time import perf_counter as timer

start_time = timer()
dot_score = util.dot_score(a = query_embedding , b = embeddings)[0]
end_time = timer()

print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds")

# 4. Get the top-k results (we'll keep top 5)
top_results = torch.topk(dot_score , k = 5)
top_results

In [ ]:
pages_and_chunks[42]

In [27]:
import textwrap

def print_wrapped(text , wrap_length = 80):
    wrapped_text = textwrap.fill(text , wrap_length)
    print(wrapped_text)


In [ ]:
query = "good food for protein"
print(f"Query: {query}\n")
print("Results:")
# Loop through zipped together scores and indices from torch.topk
for score , idx in zip(top_results[0] , top_results[1]):
    print(f"Score: {score:.4f}")
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    print(f"Page Number: {pages_and_chunks[idx]['page_number']}")
    print("\n")

In [ ]:
import fitz # pymupdf Library

# open PDF and Load target
pdf_path = pdf_path
doc = fitz.open(pdf_path)
page = doc.load_page(411 + 41) # note: page numbers of our PDF start 41+

# Get the image of the page
img = page.get_pixmap(dpi = 300)

# Save image (optional)
# img.save("output_filename.png")
doc.close()

# Convert the pixmap to a numpy array
img_array = np.frombuffer(img.samples_mv , dtype=np.uint8).reshape(img.h , img.w , img.n)


# Display the image using MatPlotlib
import matplotlib.pyplot as plt
plt.figure(figsize=(13 , 10))
plt.imshow(img_array)
plt.title(f"Query: {query} | Most relevant page:")
plt.axis("off")
plt.show()

### Similarity measures: Dot Product and Cosine similarity

In [ ]:
import torch

def dot_products(vector1 , vector2):
    return torch.dot(vector1 , vector2)

def cosine_similarity(vector1 , vector2):
    dot_product = torch.dot(vector1 , vector2)

    # Get Eucludian/L2 norm
    norm_vector1 = torch.sqrt(torch.sum(vector1**2))
    norm_vector2 = torch.sqrt(torch.sum(vector2**2))


    return dot_product / (norm_vector1 * norm_vector2)

# Examples vectors/tensors
vector1 = torch.tensor([1 , 2 , 3] , dtype=torch.float32)
vector2 = torch.tensor([1 , 2 , 3] , dtype=torch.float32)
vector3 = torch.tensor([4 ,5 , 6] , dtype=torch.float32)
vector4 = torch.tensor([-1 , -2 , -3] , dtype=torch.float32)

# Calculate dot product
print("Dot Product between vector1 and vector2 " , dot_products(vector1 , vector2))
print("Dot Product between vector1 and vector3 " , dot_products(vector1 , vector3))
print("Dot Product between vector1 and vector4 " , dot_products(vector1 , vector4))

# Calculate cosine similarity
print("Cosine Similarity between vector1 and vector2 " , cosine_similarity(vector1 , vector2))
print("Cosine Similarity  between vector1 and vector3 " , cosine_similarity(vector1 , vector3))
print("Cosine Similarity  between vector1 and vector4 " , cosine_similarity(vector1 , vector4))

### Functionizing our semantic search pipeline

In [31]:
def retrieve_relevant_resources(query: str , embeddings: torch.tensor , model: SentenceTransformer = embedding_model , n_resources_to_return : int = 5 , print_time: bool = True):
    """Embeds a query with model and returns top k scores and indices from embeddings. """

    # Embed the query
    query_embedding = model.encode(query , convert_to_tensor=True)
    query_embedding = query_embedding.to("mps")

    embeddings = embeddings.to("mps")

    # Get dot products scores on embeddings
    start_time = timer()
    dot_scores = util.dot_score(query_embedding , embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on ({len(embeddings)}) embeddings: {end_time - start_time:.5f} seconds.")
    
    scores , indices = torch.topk(input=dot_scores , k = n_resources_to_return)

    return scores , indices

def print_top_results_and_scores(query: str , embeddings: torch.tensor , pages_and_chunks: list[dict] = pages_and_chunks , n_resources_to_return: int = 5):
    """
    Finds relevant passages given a query and prints them out along with their scores. 
    """
    scores , indices = retrieve_relevant_resources(query=query , embeddings=embeddings , n_resources_to_return=n_resources_to_return)

    # Loop through zipped together scores and indices from torch.topk
    for score , idx in zip(scores ,indices):
        print(f"Score: {score:.4f}")
        print("Text:")
        print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
        print(f"Page Number: {pages_and_chunks[idx]['page_number']}")
        print("\n")


In [ ]:
print_top_results_and_scores(query="foods high in fiber" , embeddings=embeddings)

### Getting an LLM for Local generations

In [33]:
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb}GB")

In [34]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False 
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")


### Loading an LLM locally

In [ ]:
import torch
from transformers import AutoTokenizer , AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available

# 1. Create a quantization config
# Note: requires !pip install bitsandbytes accelerate
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True , bnb_4bit_compute_dtype=torch.float16)


# Bonus: flash attention 2 = faster attention mechanism
# Flash Attention 2 requires a GPU with a compute capability score of 8.0+
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability()[0] >= 8):
    attn_implementation = "flash_attention_2"
else:
    attn_implementation = "sdpa" # Scaled dot product attention

# 2. Pick a model we'd like to use
model_id = "google/gemma-7b-it"
model_id = model_id

# 3. Instantiate tokenizer (tokenizer turns text into tokens)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

# 4. Instantiate the model
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id , torch_dtype = torch.float16 , 
                                                 quantization_config = quantization_config if use_quantization_config else None , 
                                                 low_cpu_mem_usage = False , 
                                                 attn_implementation = attn_implementation)



if not use_quantization_config:
    llm_model.to("cuda")

In [ ]:
# pip install bitsandbytes accelerate

In [ ]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

In [ ]:
def get_model_mem_size(model: torch.nn.Module):
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate model sizes
    model_mem_bytes = mem_params + mem_buffers
    model_mem_mb = model_mem_bytes / (1024**2)
    model_mem_gb = model_mem_bytes / (1024**3)

    return {"model_mem_bytes": model_mem_bytes , 
            "model_mem_mb": round(model_mem_mb , 2) , 
            "model_mem_gb": round(model_mem_gb , 2)}


get_model_mem_size(llm_model)

### Generating text with our LLM

In [ ]:
input_text = "What are the macronutrients, and what roles do they play in the human body?"
print(f"Input text : \n{input_text}")

# Create prompt template for instruction-tuned model
dialogue_template = [
    {"role":"user" , 
     "content": input_text}
]

# Apply the chat template
prompt = tokenizer.apply_chat_template(conversation=dialogue_template , 
                                       tokenize=False , 
                                       add_generation_prompt=True)

print(f"\n Prompt (formated): \n {prompt} ")

In [ ]:
%%time

# Tokenize the input text (turn it into numbers) and send it to the GPU
input_ids = tokenizer(prompt , 
    return_tensors = "pt").to("cuda")

# Generate outputs from local LLM
outputs = llm_model.generate(**input_ids , max_new_tokens = 256)

print(f"Model output (toknes): \n {outputs[0]}\n")

In [ ]:
# Decode the output tokens to text
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (decoded): \n {outputs_decoded}\n")

### Augmenting our prompt with context items

In [ ]:
def prompt_formatter(query:str , 
                     context_items: list[dict]) -> str:
    context = "_ " + 